# Climatology plots for different variables
### Created by Marisol, 6 June 2022

In [ ]:
import warnings 
warnings.simplefilter('ignore') 

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 
#import fsspec # these libraries help reading cloud data
#import s3fs
#import dask
#from dask.distributed import performance_report, Client, progress
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature

In [ ]:
lat1, lat2 = 17, 20 
lon1, lon2 = -89,-86.5

In [ ]:
def plot_seas(dsa, tit, vm, vx):
    fig=plt.figure(figsize=(5,3),dpi=150)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([lon1,lon2,lat1,lat2],crs=ccrs.PlateCarree())
    ax.set_xticks([*range(int(lon1),int(lon2))], crs=ccrs.PlateCarree())
    ax.set_yticks([*range(int(lat1),int(lat2+1))], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.LAND, color='lightgrey')
    dsa.plot(cmap='jet', vmin=vm, vmax=vx)
    plt.plot(-87.71,18.72,'m*')
    plt.plot(-87.83,18.27,'m*')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(tit)
    plt.show()
    
def seas_clim(dsx, var, timevar, off=[0]):
    # seasons
    # secas: feb-may
    # lluvias: jun-sep
    # norte: oct-jan
    a = (dsx[timevar].dt.month>=2)&(dsx[timevar].dt.month<=5)
    secas = dsx[var][a].mean(dim={timevar},keep_attrs=True) + off
    a = (dsx[timevar].dt.month>=6)&(dsx[timevar].dt.month<=9)
    lluvias = dsx[var][a].mean(dim={timevar},keep_attrs=True) + off
    a = (dsx[timevar].dt.month==1)|(dsx[timevar].dt.month>=10)
    norte = dsx[var][a].mean(dim={timevar},keep_attrs=True) + off

    vm = np.min([secas.min().values,lluvias.min().values,norte.min().values])
    vx = np.max([secas.max().values,lluvias.max().values,norte.max().values])
    plot_seas(secas, 'Estacion Seca (Feb-May)', vm, vx)
    plot_seas(lluvias, 'Estacion Lluvias (Jun-Sep)', vm, vx)
    plot_seas(norte, 'Estacion Norte (Oct-Jan)', vm, vx)

In [ ]:
# Low resolution SST

ds = xr.open_mfdataset('../data/OISST/*.nc')
ds.close()

seas_clim(ds,'sst','time')
ds

In [ ]:
# Air temp

ds = xr.open_mfdataset('../data/ERA5_AT2m/*.nc')
ds.close()

seas_clim(ds,'air_temperature_at_2_metres','time0',-273.15)
ds

In [ ]:
# Precipitation

ds = xr.open_mfdataset('../data/ERA5_Rain/*.nc')
ds.close()

seas_clim(ds,'precipitation_amount_1hour_Accumulation','time1')
ds

In [ ]:
# Solar Radiation

ds = xr.open_mfdataset('../data/ERA5_Rad/*.nc')
ds.close()

seas_clim(ds,'integral_wrt_time_of_surface_direct_downwelling_shortwave_flux_in_air_1hour_Accumulation','time1')
ds

# currents

In [ ]:
dsx = xr.open_dataset('../data/Glorys_Currents/GLORYS_1993-01-16_to_2019-12-16_variable_10-0mcurrents.nc')
dsx.close()
timevar='time'
dsx

In [ ]:
# Calculate seasonal values
var = 'uo'
a = (dsx[timevar].dt.month>=2)&(dsx[timevar].dt.month<=5)
secas_u = dsx[var][a].mean(dim={timevar},keep_attrs=True) 
a = (dsx[timevar].dt.month>=6)&(dsx[timevar].dt.month<=9)
lluvias_u = dsx[var][a].mean(dim={timevar},keep_attrs=True)
a = (dsx[timevar].dt.month==1)|(dsx[timevar].dt.month>=10)
norte_u = dsx[var][a].mean(dim={timevar},keep_attrs=True) 

var = 'vo'
a = (dsx[timevar].dt.month>=2)&(dsx[timevar].dt.month<=5)
secas_v = dsx[var][a].mean(dim={timevar},keep_attrs=True) 
a = (dsx[timevar].dt.month>=6)&(dsx[timevar].dt.month<=9)
lluvias_v = dsx[var][a].mean(dim={timevar},keep_attrs=True)
a = (dsx[timevar].dt.month==1)|(dsx[timevar].dt.month>=10)
norte_v = dsx[var][a].mean(dim={timevar},keep_attrs=True)

secas = xr.Dataset(
    data_vars=dict(
        uo = (['latitude','longitude'], secas_u.data),
        vo = (['latitude','longitude'], secas_v.data),
    ),
    coords = dict(
        longitude = secas_u.longitude,
        latitude = secas_v.latitude,
    ))
lluvias = xr.Dataset(
    data_vars=dict(
        uo = (['latitude','longitude'], lluvias_u.data),
        vo = (['latitude','longitude'], lluvias_v.data),
    ),
    coords = dict(
        longitude = lluvias_u.longitude,
        latitude = lluvias_v.latitude,
    ))
norte = xr.Dataset(
    data_vars=dict(
        uo = (['latitude','longitude'], norte_u.data),
        vo = (['latitude','longitude'], norte_v.data),
    ),
    coords = dict(
        longitude = norte_u.longitude,
        latitude = norte_v.latitude,
    ))

secas['spd'] = np.sqrt(secas.uo**2+secas.vo**2)
lluvias['spd'] = np.sqrt(lluvias.uo**2+lluvias.vo**2)
norte['spd'] = np.sqrt(norte.uo**2+norte.vo**2)

vm = np.min([secas['spd'].min().values,lluvias['spd'].min().values,norte['spd'].min().values])
vx = np.max([secas['spd'].max().values,lluvias['spd'].max().values,norte['spd'].max().values])

In [ ]:

def plot_vseas(dsa, tit, vm, vx):
    fig=plt.figure(figsize=(5,3),dpi=150)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([lon1,lon2,lat1,lat2],crs=ccrs.PlateCarree())
    ax.set_xticks([*range(int(lon1),int(lon2))], crs=ccrs.PlateCarree())
    ax.set_yticks([*range(int(lat1),int(lat2+1))], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.LAND, color='lightgrey')
    
    dsa['spd'].plot(cmap='jet', vmin=vm, vmax=vx, alpha=0.5)
    dsa.plot.quiver('longitude','latitude','uo','vo')
    plt.plot(-87.71,18.72,'m*')
    plt.plot(-87.83,18.27,'m*')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(tit)
    plt.tight_layout()
    plt.show()
    
plot_vseas(secas, 'Estacion Seca (Feb-May)', vm, vx)
plot_vseas(lluvias, 'Estacion Lluvias (Jun-Sep)', vm, vx)
plot_vseas(norte, 'Estacion Norte (Oct-Jan)', vm, vx)

# winds 

In [ ]:
ds1 = xr.open_mfdataset('../data/ERA5_MW10m/*.nc')
ds1.close()
timevar='time0'
ds2 = xr.open_mfdataset('../data/ERA5_ZW10m/*.nc')
ds2.close()
timevar='time0'
dsx = xr.Dataset(
    data_vars=dict(
        vo = (['time0','lat','lon'], ds1.northward_wind_at_10_metres.data),
        uo = (['time0','lat','lon'], ds2.eastward_wind_at_10_metres.data),
    ),
    coords = dict(
        time0 = ds1.time0,
        lon = ds1.lon,
        lat = ds1.lat,
    ))
dsx['wspd']= np.sqrt(dsx.uo**2+dsx.vo**2)
dsx

In [ ]:
var = 'uo'
a = (dsx[timevar].dt.month>=2)&(dsx[timevar].dt.month<=5)
secas_u = dsx[var][a].mean(dim={timevar},keep_attrs=True) 
a = (dsx[timevar].dt.month>=6)&(dsx[timevar].dt.month<=9)
lluvias_u = dsx[var][a].mean(dim={timevar},keep_attrs=True)
a = (dsx[timevar].dt.month==1)|(dsx[timevar].dt.month>=10)
norte_u = dsx[var][a].mean(dim={timevar},keep_attrs=True) 

var = 'vo'
a = (dsx[timevar].dt.month>=2)&(dsx[timevar].dt.month<=5)
secas_v = dsx[var][a].mean(dim={timevar},keep_attrs=True) 
a = (dsx[timevar].dt.month>=6)&(dsx[timevar].dt.month<=9)
lluvias_v = dsx[var][a].mean(dim={timevar},keep_attrs=True)
a = (dsx[timevar].dt.month==1)|(dsx[timevar].dt.month>=10)
norte_v = dsx[var][a].mean(dim={timevar},keep_attrs=True)

secas = xr.Dataset(
    data_vars=dict(
        uo = (['lat','lon'], secas_u.data),
        vo = (['lat','lon'], secas_v.data),
    ),
    coords = dict(
        lon = secas_u.lon,
        lat = secas_v.lat,
    ))
lluvias = xr.Dataset(
    data_vars=dict(
        uo = (['lat','lon'], lluvias_u.data),
        vo = (['lat','lon'], lluvias_v.data),
    ),
    coords = dict(
        lon = lluvias_u.lon,
        lat = lluvias_v.lat,
    ))
norte = xr.Dataset(
    data_vars=dict(
        uo = (['lat','lon'], norte_u.data),
        vo = (['lat','lon'], norte_v.data),
    ),
    coords = dict(
        lon = norte_u.lon,
        lat = norte_v.lat,
    ))

secas['spd'] = np.sqrt(secas.uo**2+secas.vo**2)
lluvias['spd'] = np.sqrt(lluvias.uo**2+lluvias.vo**2)
norte['spd'] = np.sqrt(norte.uo**2+norte.vo**2)

vm = np.min([secas['spd'].min().values,lluvias['spd'].min().values,norte['spd'].min().values])
vx = np.max([secas['spd'].max().values,lluvias['spd'].max().values,norte['spd'].max().values])

In [ ]:
  
def plot_vseas(dsa, tit, vm, vx):
    fig=plt.figure(figsize=(5,3),dpi=150)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([lon1,lon2,lat1,lat2],crs=ccrs.PlateCarree())
    ax.set_xticks([*range(int(lon1),int(lon2))], crs=ccrs.PlateCarree())
    ax.set_yticks([*range(int(lat1),int(lat2+1))], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.LAND, color='lightgrey')
    
    dsa['spd'].plot(cmap='jet', vmin=vm, vmax=vx, alpha=0.5)
    dsa.plot.quiver('lon','lat','uo','vo')
    plt.plot(-87.71,18.72,'m*')
    plt.plot(-87.83,18.27,'m*')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.title(tit)
    plt.tight_layout()
    plt.show()
    
plot_vseas(secas, 'Estacion Seca (Feb-May)', vm, vx)
plot_vseas(lluvias, 'Estacion Lluvias (Jun-Sep)', vm, vx)
plot_vseas(norte, 'Estacion Norte (Oct-Jan)', vm, vx)